In [ ]:
import pandas as pd
import numpy as np


In [ ]:
DATA_PATH = "./data/"
MED_FILE = "PRESCRIPTIONS.csv"
DIAGS_FILE = "DIAGNOSES_ICD.csv"
NDC_TO_ADC = "package_NDC_ATC4_classes.csv"

In [ ]:
Field = ["SUBJECT_ID","HADM_ID","ICUSTAY_ID","STARTDATE","NDC"]
FieldType = {"SUBJECT_ID": 'Int64',
             "HADM_ID": 'Int64',
             "ICUSTAY_ID": 'Int64',
             "NDC": 'str',
             }
med_df = pd.read_csv(DATA_PATH+MED_FILE, usecols=Field, dtype=FieldType, parse_dates=["STARTDATE"])
med_df = med_df[med_df['NDC'] != '0'] # shape after this is (3569864, 5)
### I'm copying their code between here, and trying to understand why they pad and dropna
# med_df.fillna(method='pad', inplace=True)
# med_df.dropna(inplace=True) # w/o med_df.head(20)
# med_df.drop_duplicates(inplace=True) #(2962482, 5) without paddinng and dropping na
###



C:\Users\kevin\AppData\Local\Temp\ipykernel_57664\2086015911.py:9: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  med_df.fillna(method='pad', inplace=True)


In [48]:
med_df.head(20)
med_df.shape

(2987441, 5)

In [ ]:
### Their code
## Process Meds
print('process_med')
med_pd = pd.read_csv(DATA_PATH+MED_FILE, dtype={'NDC':'category'})
# filter
med_pd.drop(columns=['ROW_ID','DRUG_TYPE','DRUG_NAME_POE','DRUG_NAME_GENERIC',
                    'FORMULARY_DRUG_CD','GSN','PROD_STRENGTH','DOSE_VAL_RX',
                    'DOSE_UNIT_RX','FORM_VAL_DISP','FORM_UNIT_DISP','FORM_UNIT_DISP',
                    'ROUTE','ENDDATE','DRUG'], axis=1, inplace=True)
med_pd.drop(index = med_pd[med_pd['NDC'] == '0'].index, axis=0, inplace=True)
med_pd.fillna(method='pad', inplace=True)
med_pd.dropna(inplace=True)
med_pd.drop_duplicates(inplace=True) # (2987420, 5)
# med_pd['ICUSTAY_ID'] = med_pd['ICUSTAY_ID'].astype('int64')
# med_pd['STARTDATE'] = pd.to_datetime(med_pd['STARTDATE'], format='%Y-%m-%d %H:%M:%S')    
# med_pd.sort_values(by=['SUBJECT_ID', 'HADM_ID', 'ICUSTAY_ID', 'STARTDATE'], inplace=True)
# med_pd = med_pd.reset_index(drop=True)

# def filter_first24hour_med(med_pd):
#     med_pd_new = med_pd.drop(columns=['NDC'])
#     med_pd_new = med_pd_new.groupby(by=['SUBJECT_ID','HADM_ID','ICUSTAY_ID']).head([1]).reset_index(drop=True)
#     med_pd_new = pd.merge(med_pd_new, med_pd, on=['SUBJECT_ID','HADM_ID','ICUSTAY_ID','STARTDATE'])
#     med_pd_new = med_pd_new.drop(columns=['STARTDATE'])
#     return med_pd_new
# med_pd = filter_first24hour_med(med_pd) # or next line
# #     med_pd = med_pd.drop(columns=['STARTDATE'])

# med_pd = med_pd.drop(columns=['ICUSTAY_ID'])
# med_pd = med_pd.drop_duplicates()
    
#     return med_pd.reset_index(drop=True)

process_med


C:\Users\kevin\AppData\Local\Temp\ipykernel_57664\327004468.py:3: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  med_pd = pd.read_csv(DATA_PATH+MED_FILE, dtype={'NDC':'category'})
C:\Users\kevin\AppData\Local\Temp\ipykernel_57664\327004468.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  med_pd.fillna(method='pad', inplace=True)


In [44]:
med_pd.head(20)
med_pd.shape

(2987420, 5)